In [ ]:
import numpy as np
import os
import glob
import cv2
import time
import sys

In [ ]:
def ComplexMul(x, y):
    temp = np.zeros(x.shape, y.dtype)

    temp[:, :, 0] = x[:, :, 0] * y[:, :, 0] - x[:, :, 1] * y[:, :, 1]
    temp[:, :, 1] = x[:, :, 0] * y[:, :, 1] + x[:, :, 1] * y[:, :, 0]
    return temp

In [ ]:
def ComplexDiv(x, y):
    temp = np.zeros(x.shape, y.dtype)
    divisor = 1. / (y[:, :, 0] ** 2 + y[:, :, 1] ** 2)

    temp[:, :, 0] = (x[:, :, 0] * y[:, :, 0] + x[:, :, 1] * y[:, :, 1]) * divisor
    temp[:, :, 1] = (x[:, :, 1] * y[:, :, 0] + x[:, :, 0] * y[:, :, 1]) * divisor
    return temp

In [ ]:
def FourierTrans(img, back=False):
    return cv2.dft(np.float32(img), flags=(
        (cv2.DFT_INVERSE | cv2.DFT_SCALE) if back else cv2.DFT_COMPLEX_OUTPUT))  

In [ ]:
def Rearrange(img):

    #assert (img.ndim == 2)
    img_new = np.zeros(img.shape, img.dtype)
    xh, yh = img.shape[1] // 2, img.shape[0] // 2
    img_new[0:yh, 0:xh], img_new[yh:img.shape[0], xh:img.shape[1]] = img[yh:img.shape[0], xh:img.shape[1]], img[0:yh, 0:xh]
    img_new[0:yh, xh:img.shape[1]], img_new[yh:img.shape[0], 0:xh] = img[yh:img.shape[0], 0:xh], img[0:yh, xh:img.shape[1]]
    return img_new

In [ ]:
def Limit(rect, limit):
    if (rect[0] + rect[2] > limit[0] + limit[2]):
        rect[2] = limit[0] + limit[2] - rect[0]
    if (rect[1] + rect[3] > limit[1] + limit[3]):
        rect[3] = limit[1] + limit[3] - rect[1]
    if (rect[0] < limit[0]):
        rect[2] -= (limit[0] - rect[0])
        rect[0] = limit[0]
    if (rect[1] < limit[1]):
        rect[3] -= (limit[1] - rect[1])
        rect[1] = limit[1]
    if (rect[2] < 0):
        rect[2] = 0
    if (rect[3] < 0):
        rect[3] = 0
    return rect

In [ ]:
def GetBorder(window, cut):
    res = [0, 0, 0, 0]
    res[0] = cut[0] - window[0]
    res[1] = cut[1] - window[1]
    
    res[2] = window[0]+window[2] - (cut[0]+cut[2])
    res[3] = window[1]+window[3] - (cut[1]+cut[3])
   
    return res

In [ ]:
def Subwindow(img, window, borderType=cv2.BORDER_CONSTANT):
    cutWindow = [x for x in window]
    Limit(cutWindow, [0, 0, img.shape[1], img.shape[0]])  
    assert (cutWindow[2] > 0 and cutWindow[3] > 0)
    border = GetBorder(window, cutWindow)
    res = img[cutWindow[1]:cutWindow[1] + cutWindow[3], cutWindow[0]:cutWindow[0] + cutWindow[2]]

    if (border != [0, 0, 0, 0]):
        res = cv2.copyMakeBorder(res, border[1], border[3], border[0], border[2], borderType)
    return res

In [ ]:
# KCF tracker
class KCFTracker:
    def __init__(self, hog=False, fixed=False, multiscale=True):
        self.lambdar = 0.001  # regularization term
        # padding might be slightly different for each dataset
        self.padding =  2.5
        self.output_sigma_const= 0.125  # bandwidth of gaussian 

        if (hog):  
            self.interp_factor = 0.075  
            self.sigma = 0.2 
            self.cell_size = 8  
            self.hog_feature = True
        else:  
            self.interp_factor = 0.075
            self.sigma = 0.2
            self.cell_size = 1
            self.hog_feature = False

        if (multiscale):
            self.template_size = 96  
            self.scale_step = 1.1  
            self.scale_weight = 0.96  
            if(not fixed):
                fixed = True
        elif (fixed):
            self.template_size = 96
            self.scale_step = 1
        else:
            self.template_size = 1
            self.scale_step = 1

        self.temp_size = [0, 0] 
        self.roi = [0., 0., 0., 0.]  
        self.size_patch = [0, 0, 0] 
        self.scale = 1.0
        self.alpha = None  
        self.prob = None  
        self.temp = None 
        self.hann = None 

    def PixelPeak(self, left, center, right):
        divisor = 2 * center - right - left  
        return (0 if abs(divisor) < 1e-3 else 0.5 * (right - left) / divisor)

    def HanningMats(self):
        hann2t, hann1t = np.ogrid[0:self.size_patch[0], 0:self.size_patch[1]]
        hann1t = 0.5 * (1 - np.cos(2 * np.pi * hann1t / (self.size_patch[1] - 1)))
        hann2t = 0.5 * (1 - np.cos(2 * np.pi * hann2t / (self.size_patch[0] - 1)))
        hann2d = hann2t * hann1t

        if (self.hog_feature):
            hann1d = hann2d.reshape(self.size_patch[0] * self.size_patch[1])
            self.hann = np.zeros((self.size_patch[2], 1), np.float32) + hann1d      
        else:
            self.hann = hann2d
        self.hann = self.hann.astype(np.float32)

    def GaussianPeak(self, sizey, sizex):                                     
        syh, sxh = sizey / 2, sizex / 2
        sigma = np.sqrt(sizex * sizey) / self.padding * self.output_sigma_const
        mult = -0.5 / (sigma * sigma)
        y, x = np.ogrid[0:sizey, 0:sizex]
        y, x = (y - syh) ** 2, (x - sxh) ** 2
        res = np.exp(mult * (y + x))
        return FourierTrans(res)

    def GaussianCorrelation(self, x1, x2):
        if (self.hog_feature):
            c = np.zeros((self.size_patch[0], self.size_patch[1]), np.float32)
            for i in xrange(self.size_patch[2]):
                x1_aux = x1[i, :].reshape((self.size_patch[0], self.size_patch[1]))
                x2_aux = x2[i, :].reshape((self.size_patch[0], self.size_patch[1]))
                caux = cv2.mulSpectrums(FourierTrans(x1_aux), FourierTrans(x2_aux), 0, conjB=True)
                caux = FourierTrans(caux, True)[:, :, 0]
                c += caux
            c = Rearrange(c)
        else:
            c = cv2.mulSpectrums(FourierTrans(x1), FourierTrans(x2), 0, conjB=True)  
            c = FourierTrans(c, True)
            c = c[:, :, 0]
            c = Rearrange(c)

        if (x1.ndim == 3 and x2.ndim == 3):
            d = (np.sum(x1[:, :, 0] * x1[:, :, 0]) + np.sum(x2[:, :, 0] * x2[:, :, 0]) - 2.0 * c) / (
                        self.size_patch[0] * self.size_patch[1] * self.size_patch[2])
        elif (x1.ndim == 2 and x2.ndim == 2):
            d = (np.sum(x1 * x1) + np.sum(x2 * x2) - 2.0 * c) / (
                        self.size_patch[0] * self.size_patch[1] * self.size_patch[2])

        d = d * (d >= 0)
        d = np.exp(-d / (self.sigma * self.sigma))

        return d

    def GetFeatures(self, image, inithann, scale_adjust=1.0):
        extracted_roi = [0, 0, 0, 0]  
        cx = self.roi[0] + self.roi[2] / 2    
        cy = self.roi[1] + self.roi[3] / 2 

        if (inithann):
            padded_w = self.roi[2] * self.padding
            padded_h = self.roi[3] * self.padding

            if (self.template_size > 1):
                if (padded_w >= padded_h):
                    self.scale = padded_w / float(self.template_size)    
                else:
                    self.scale = padded_h / float(self.template_size)    
                self.temp_size[0] = int(padded_w / self.scale)
                self.temp_size[1] = int(padded_h / self.scale)
            else:
                self.temp_size[0] = int(padded_w)
                self.temp_size[1] = int(padded_h)
                self.scale = 1.0

            if self.hog_feature:
                self.temp_size[0] = int(self.temp_size[0]) // (
                            2 * self.cell_size) * 2 * self.cell_size + 2 * self.cell_size
                self.temp_size[1] = int(self.temp_size[1]) // (
                            2 * self.cell_size) * 2 * self.cell_size + 2 * self.cell_size
            else:
                self.temp_size[0] = int(self.temp_size[0]) // 2 * 2
                self.temp_size[1] = int(self.temp_size[1]) // 2 * 2

        
        extracted_roi[2] = int(scale_adjust * self.scale * self.temp_size[0])       
        extracted_roi[3] = int(scale_adjust * self.scale * self.temp_size[1])
        extracted_roi[0] = int(cx - extracted_roi[2] / 2)
        extracted_roi[1] = int(cy - extracted_roi[3] / 2)
        z = Subwindow(image, extracted_roi, cv2.BORDER_REPLICATE)                       
        if (z.shape[1] != self.temp_size[0] or z.shape[0] != self.temp_size[1]):         
            z = cv2.resize(z, tuple(self.temp_size))

        if (self.hog_feature):
            FeaturesMap = z
            FeaturesMap = FeaturesMap.astype(np.float32) / 255.0 - 0.5 
            self.size_patch = [z.shape[0], z.shape[1], z.shape[2]]  
            FeaturesMap=FeaturesMap.reshape((self.size_patch[0] * self.size_patch[1],
                                               self.size_patch[2])).T
        else:                                                     
            if (z.ndim == 3 and z.shape[2] == 3):
                FeaturesMap = cv2.cvtColor(z,cv2.COLOR_BGR2GRAY) 
            elif (z.ndim == 2):
                FeaturesMap = z  
            FeaturesMap = FeaturesMap.astype(np.float32) / 255.0 - 0.5          
            self.size_patch = [z.shape[0], z.shape[1], 1]                      

        if (inithann):
            self.HanningMats()  

        FeaturesMap = self.hann * FeaturesMap                                  
        return FeaturesMap

   

    def Detect(self, z, x):                                            
        k = self.GaussianCorrelation(x, z)
        res = FourierTrans(ComplexMul(self.alpha, FourierTrans(k)), True)[:, :, 0]         

        _, pv, _, pi = cv2.minMaxLoc(res)  
        p = [float(pi[0]), float(pi[1])]  
        if (pi[0] > 0 and pi[0] < res.shape[1] - 1):
            p[0] += self.PixelPeak(res[pi[1], pi[0] - 1], pv, res[pi[1], pi[0] + 1])  
        if (pi[1] > 0 and pi[1] < res.shape[0] - 1):
            p[1] += self.PixelPeak(res[pi[1] - 1, pi[0]], pv, res[pi[1] + 1, pi[0]])

        p[0] -= res.shape[1] / 2.
        p[1] -= res.shape[0] / 2.
                                                                                        
        return p, pv                                                                   

    def Train(self, x, train_interp_factor):
        k = self.GaussianCorrelation(x, x)
        alphaf = ComplexDiv(self.prob, FourierTrans(k) + self.lambdar)                   
                                                                                      
        self.temp = (1 - train_interp_factor) * self.temp + train_interp_factor * x          
        self.alpha = (1 - train_interp_factor) * self.alpha + train_interp_factor * alphaf    
   


    def Init(self, roi, image):
        self.roi = list(map(float,roi))
        #assert (roi[2] > 0 and roi[3] > 0)
        self.temp = self.GetFeatures(image, 1)                                                 
        self.prob = self.GaussianPeak(self.size_patch[0], self.size_patch[1])           
        self.alpha = np.zeros((self.size_patch[0], self.size_patch[1], 2), np.float32)       
        self.Train(self.temp, 1.0)
   

    def Update(self, image):
    
        if (self.roi[0] + self.roi[2] <= 0):  self.roi[0] = -self.roi[2] + 1          
        if (self.roi[1] + self.roi[3] <= 0):  self.roi[1] = -self.roi[3] + 1
        if (self.roi[0] >= image.shape[1] - 1):  self.roi[0] = image.shape[1] - 2
        if (self.roi[1] >= image.shape[0] - 1):  self.roi[1] = image.shape[0] - 2
        cx = self.roi[0] + self.roi[2] / 2.      
        cy = self.roi[1] + self.roi[3] / 2.

        loc, peak_value = self.Detect(self.temp, self.GetFeatures(image, 0, 1.0))
        if (self.scale_step != 1):
            new_loc, new_peak_value = self.Detect(self.temp, self.GetFeatures(image, 0, 1.0 / self.scale_step))
            if (1.06* new_peak_value > peak_value):
                loc = new_loc
                peak_value = new_peak_value
                self.scale /= self.scale_step
                self.roi[2] /= self.scale_step
                self.roi[3] /= self.scale_step

            new_loc, new_peak_value = self.Detect(self.temp, self.GetFeatures(image, 0, self.scale_step))

            if (self.scale_weight * new_peak_value > peak_value):
                loc = new_loc
                peak_value = new_peak_value
                self.scale *= self.scale_step
                self.roi[2] *= self.scale_step
                self.roi[3] *= self.scale_step

        self.roi[0] = cx - self.roi[2] / 2.0 + loc[0] * self.cell_size * self.scale        
        self.roi[1] = cy - self.roi[3] / 2.0 + loc[1] * self.cell_size * self.scale

        if (self.roi[0] >= image.shape[1] - 1):  self._roi[0] = image.shape[1] - 1
        if (self.roi[1] >= image.shape[0] - 1):  self._roi[1] = image.shape[0] - 1
        if (self.roi[0] + self.roi[2] <= 0):  self.roi[0] = -self.roi[2] + 2
        if (self.roi[1] + self.roi[3] <= 0):  self.roi[1] = -self.roi[3] + 2
        assert (self.roi[2] > 0 and self.roi[3] > 0)
        
        x = self.GetFeatures(image, 0, 1.0)
        self.Train(x, self.interp_factor)
        
        return peak_value, self.roi

In [ ]:
if __name__ == '__main__' :
    
    img_dir = "./BlurCar2/img"  
    data_path = os.path.join(img_dir, "*.jpg") 
    img_files = glob.glob(data_path)
    img_files.sort()
    video_name = 'BlurCar2.mp4'
    
    frame = cv2.imread(img_files[0])
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'DIVX'), 15, (width,height))
    
    for image in img_files:
        video.write(cv2.imread(image))
    video.release()
    
    tracker = KCFTracker()
    video = cv2.VideoCapture(video_name)
    _,frame = video.read()
    bbox = cv2.selectROI(frame, False)
    frames = 0
    
    start = time.time()
    tracker.Init(bbox,frame)
    
    while True:
        _,frame = video.read();
        if(_):
            peak_value, item = tracker.Update(frame);
            frames += 1
           
            x = int(item[0])
            y = int(item[1])
            w = int(item[2])
            h = int(item[3])
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.imshow("tracking",frame)
            k = cv2.waitKey(1) & 0xff
            if k == 27:
                break